In [ ]:
from email import message_from_string
import pandas as pd
import re

In [ ]:
df = pd.read_csv("./dataset/Processed_datasets/spamassassin_dataset.csv")

def extract_mail_components(text):
    try:
        msg = message_from_string(text)
        m_body = msg.get_payload()
        m_from = msg.get('From', '')
        m_return = msg.get('Return-Path', '')

        re_find = r'<([^>]*)>' 

        m_from = re.findall(re_find, m_from)[0] if re.findall(re_find, m_from) else ''
        m_return = re.findall(re_find, m_return)[0] if re.findall(re_find, m_return) else ''

        m_body = str(m_body)
        m_from = str(m_from)
        m_return = str(m_return)

        return m_body, m_from, m_return
    except Exception as e:
        print(f"Error processing text: {text[:100]}... Error: {e}")
        return '', '', ''  # Return empty values if there is an error processing the text

def extract_urls(text):
    import re
    url_list = []
    ipv4_pattern = r'(?:https?|ftp)?://(?:\d{1,3}\.){3}\d{1,3}(?::\d+)?(?:/\S*)?'
    url_pattern = (
        r'(?:(?:https?|ftp):\/\/)?'
        r'(?:[a-zA-Z0-9.%+-]+:[a-zA-Z0-9.%+-]+@)?'
        r'(?:www\.)?[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
        r'(?:\/\S*)?'
        r'|'
        + ipv4_pattern
    )
    email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'

    # Split the input string into words
    words = text.split()

    for word in words:
        if re.fullmatch(email_pattern, word):
            # Check if it also matches a URL and additional conditions
            if re.fullmatch(url_pattern, word):
                if word.count('@') > 1 or ":" in word.split('@')[-1]:
                    url_list.append(word)
        elif re.fullmatch(url_pattern, word):
            url_list.append(word)

    return url_list


df[['Body', 'From', 'Return-Path']] = df['Content'].apply(extract_mail_components).apply(pd.Series)
df['Urls'] = df['Body'].apply(extract_urls)
